High-level overview Description of algorithms 
Description of data structures 
Database design
Design of user interface


Hardware design/selection (if appropriate) 


System security and integrity of data For an investigative project; a log of stages taken during 
the investigation and how this affected the coding

# Design

## Overall System Design

The SLAP (Self Logging Auto Pilot) system is designed to provide automated steering control for small sailing vessels while maintaining detailed logs of each journey. The system consists of several interconnected components that work together to achieve reliable navigation.

At its core, the system uses a PID (Proportional-Integral-Derivative) control loop that continuously monitors the boat's heading and makes adjustments to maintain the desired course. This is accomplished through a group of sensors including GPS for position and speed data, and motion sensors to detect the boat's movement and sea conditions. The sensor data is processed in real-time to calculate necessary rudder adjustments.

The user interface is implemented as a web application built with Flask, allowing sailors to interact with the system through any mobile device or computer with a web browser. This interface provides visualisation of key data like current heading, speed, and wind conditions, whilst also allowing manual control inputs when needed. The web application communicates with a rasberry pi based webserver over a local WiFi connection.

Data logging is a key feature of the system, with detailed records of each journey stored in a cloud database. The logs capture comprehensive sensor readings, including GPS coordinates, speed, motion sensor data, and control adjustments. This historical data can be viewed visually on a map

The system is designed to be reliable, it operates autonomously while still giving the sailor full control when needed.

## Modular Design

The system architecture follows a modular design approach, with separate components handling specific functions. The webserver module, running MicroPython on an Rasberry Pi or similar microcontroller, manages the file system and handles communication between the user interface and control systems. A dedicated PID module executes the main feedback control loop, receiving setpoint adjustments from the webserver when manual changes are made.

The system will be split into multiple smaller modules, a webserver running micro-python on an Arduino (or similar) will host a file system to store the user interface.

The client (Phone) will run the interface and send requests the webserver upon button press. The client will also be sending a �rest call� to update the GUI constantly over Wi-Fi.

The user interface will be written in JavaScript with React to run a HTML web application.

A separate PID module will be running custom code which preforms the main feedback loop, the webserver will communicate to this module to execute manual changes by sending a set point position.

**Modular Design Diagram:**

![modularDesign](modularDesign.drawio.png)


## Module Descriptions and Interconnections

The SLAP system consists of modules that work together to provide automated steering control. Each module has specific responsibilities and I have tried to make sure each module is responsible for its own functions. The modules are described below. The code (see technical solution) is organised along the lines of these modules,

### Auto Pilot Module
- **PID Controller**: The central control module that implements the proportional-integral-derivative control algorithm
- **Input**: Current heading, desired heading, sensor data
- **Output**: Rudder position commands
- **Interfaces**: 
  - Receives sensor data from Sensor register
  - Sends control commands to Tiller Actuator
  - Receives target heading updates from Web Interface

### Boat Sim Module
- **Components**: Virtual Boat Dynamics, Simulated geographic position
- **Responsibilities**: For testing purposes by mimicing the behaviour of a boat
- **Interfaces**:
  - Recieves rudder angle
  - Calculates new heading and position
  - Provides data to GPS (in sim mode)

### Sensor Register Module
- **Components**: GPS, Motion Sensors, Eviroment Sensors
- **Responsibilities**: A register of all the hardware transducer modules and the sensors that these contain
- **Interfaces**:
  - Sends sensor data to Auto Pilot Controller
  - Keeps the GPS module updated
  - Provides data to Logging Module
  - Communicates with Web Interface for display

### GPS Module
- **Components**: Register of GPS values
- **Responsibilities**: To provide information about the boats position and heading
- **Interfaces**:
  - Collects data from either a GPS Hardware Module, the Magnetometer or the boat simulator
  - Supplies modules which require heading and positional data

### Tiller Actuator Module
- **Components**: Servo Motor Controller
- **Responsibilities**: Converts the tiller angle into signals for the servo motor
- **Interfaces**:
  - Receives tiller angle signal from PID Controller
  - Provides feedback to Web Interface

### Web Interface Module
- **Components**: Flask Server, Javascripts, HTML Template for each page, style sheets
- **Responsibilities**: User interaction for system control and monitoring
- **Interfaces**:
  - Sends manual control inputs to Auto Pilot
  - Receives data from all sensors
  - Displays system status

### Logging Module
- **Components**: Log Time Keeper
- **Responsibilities**: Collecting data and writing to database
- **Interfaces**:
  - Receives data from all sensors
  - Talks to SlapStore database service to store logs

#### SlapStore Module
- **Components**: Database connection, Table Definitions, Storage Service Layer
- **Responsibilities**: Reading and writing to the database, keeping the database details hidden from the rest of the program
- **Interfaces**:
  - Connection to SQLite database
  - Storage services for all parts of the system#

### Mapping Module
- **Components**: Map Manager, Cloud interfacing
- **Responsibilities**: Read log data and write to the cloud mapping service
- **Interfaces**:
  - Receives data from SlapStore service layer
  - Talks to MapBox to upload trip logs

### Real-Time Behaviour
It was not possible to make the system function properly with a single loop where each componant is run in sequence. Therefore the system makes use of a number of execution threads. Each componant which needs to execute in its own loop is given a dedicated thread. The choice of which modules required a thread was based on the real world functions required.

The threaded modules are:
- Boat sim
- All sensors
- Logger
- Web server


### Communication Flow
1. Sensor Modules continuously collects data
2. The auto Pilot receives heading data and calculates control commands using the PID module
3. Actuator Module recieves the control signal from the auto pilot and  controls the servo motor
4. Logging Module records all sensor data in a regular time interval
5. Web Interface recieves user interactions and displays system status
6. Web Server, recieves interactions from the interface and sends them to the system
7. The Map Manager takes the trip log and sends it to the cloud

## Use Cases
Based on the user story and system objectives the necessary use cases to meet the overall requirements have been designed. These use cases are shown in the diagram below:

![use cases](slapUseCases.drawio.png)


## UI Navigation and Design
Bases on the user story and use cases above, the following user interface structure was designed

![userNavigationDesign](navigationDesign.drawio.png)


### UI Design

#### Index

A central value gives the angle of the target heading in degrees between 0 and 359. Central at the bottom is a set button which begins the Auto Pilot which will try to keep the boat on course. Another button will start the logging. The stop button will stop the Auto Pilot control loop. The simulate button will engage the boat sim. Manual adjustments can be made with the buttons at the top, allowing for both fine and broad adjustments. The menu button in the top left will give you access to the other pages.


## Code Base

### Object Oriented Programming

The design of the code for using hardware modules each with a number sensors is complex. To achieve a clean code solution a class inheritance pattern is utilised. In addition a [factory pattern] was also used. 

The diagram below shows the object oriented class structure:

![sensorStructure](sensorClassStruct.drawio.png)
The class structure diagram shows the inheritance hierarchy for the sensor system:

At the base level is the `Transducer` abstract class represents hardware devices. Note that each hardware device can contain a number of actual sensors. 
This class has:
- A list of sensors
- Thread management so that the readings from the hardware can be collected in real-time

The `Sensor` abstract class represents the sensors in each hardware module (Transducer) module:
- Basic sensor properties (name, units)
- getValue method to retrieve sensor reading

Specific sensor class which inherit from `Sensor` and represent real sensors.
- `heading`: The direction of travel in degrees
- `position`: The longitude and latitude
- `temperature`: The enviroment's temperature
- `pressure`: Air pressure 

This object oriented class structure allows the rest of the system to handle sensor information without having to handle their specific types. It allows a register of sensors to be created.

### Data Dictionary

Primary Sensor Data

GPS will be used to get positional data about the boat and its location relative to the desired destination using its longitude and latitude. GPS will also be used to determine approximate speed and direction using the standard NMEA format. Speed and direction will be stored as a decimal value

A Motion Sensor will be used to report information about the boat's movement and by extension the seas condition. It will possess a decimal value for each axis.

Log files will include many different data types, such as time (integer) and distance (integer) and error value which will be stored as a decimal value (float)

Data Collection

Current Time will be stored for each reading of each sensor.

The destination will be retrieved from the user and stored as a longitude and latitude coordinate (integers).

Actual course will be stored as a change in longitude and latitude (integers).

Desired course will be stored as a change in longitude and latitude (integers) from start position to desired destination.

Actuator value will be stored as an integer value.

| Data | Format | Description |
| --- | --- | --- |
| Coordinate | Array, Integer | 2 Integer values for longitude and latitude |
| Speed | Float | Value of speed in knots |
| Wind | Float | Value of wind speed in knots |
| Time | Time | Time at read |
| Motion Sensor | Float | Value of motion sensor as a decimal |
| Actuator value | Integer | The value sent to the motor for steering after processing |
| Distance | Float | Value of distance in nautical miles |


## Data Stucture Definitions


## Database Design

The database consists of four main tables:

Config Table:
- Contains parameters used to tune the control algorithm
- Each parameter has a name, value and description
- Parameters include PID control values and other tuning constants
- Multiple config sets can be stored for different conditions

Sensors Table:
- Maintains a registry of all sensors connected to the system
- Each sensor has a unique ID, name, type and calibration data
- Sensor types include GPS, compass, wind speed/direction, accelerometer etc.
- Status field tracks if sensor is active/inactive

Trips Table:
- Records details of each sailing trip/passage
- Contains start/end times, start/end locations
- Stores target heading and actual course taken
- Links to config settings used for that trip
- Performance metrics and statistics for the trip

Readings Table:
- Stores all sensor readings during trips
- Each reading links to a specific trip and sensor
- Includes timestamp and raw sensor value
- One reading record per sensor per measurement interval
- Used for analysis and performance optimization


## Data Validation

Data entered into input fields such as editing configs must be checked to ensure they are of the of the correct format and type. Being an active with multiple hardware sensors, most of the data in the system is recieved automatically. The limited user input data is validated in the user interface. The following user interfaces capture data and therefore be validated as indicated below:
| Form | Input Field | Data Type | Valid Range/Format | Description |
|------|------------|-----------|-------------------|-------------|
| Index | Set Heading | Integer | 0 to 359 degrees | Desired boat heading |
| Edit Config | Config Name | Text | 15 characters max | Name of configuration |
| Edit Config | P Value | Float | Positive decimal | Proportional control value |
| Edit Config | I Value | Float | Positive decimal | Integral control value |
| Edit Config | D Value | Float | Positive decimal | Derivative control value |


Input data from the user will not need to be validated as the only inputs are buttons of which have a predetermined function.



## Algorithms

The following sections details the two central algorithms implemented in SLAP.

### Boat Simulator

The boat simulator allows SLAP to run without the need of a real boat, this is necessary for test and development purposes, and understanding the boat dynamics and creating modelling code is a key part of the project.

There are two element to the boat simulator.The first is the mathmatical model which represents the boats response to the tiller. This is characterised by calculating the new heading using all the previous information about the boats heading. The dynamics of this turning behaviour are that the boat begins to turn quickly, as the boat turns its rate of turn slows until a steady rate is reached.

The second element of the boat modeling is the disturbances to the boats heading. The boat simulator would not be realistic as the control algorithm would simply and quickly achieve the correct heading. The boat simulator model randomly generates disturbances which affects the boats heading and forces the Auto Pilot to react.

#### Mathmatics

In order to control the movement and direction of the boat we must first be able to model how the rudders angle affects the boats direction

We understand that the rudders angle and boats direction do not respect a linear relationship, if we represent the rudder's angle with $\theta$, and the turning radius of the boat with R we can see that: $R = \frac{1}{K |\theta|} $

This inversely proptional relationship is proven as we know as the rudders angle increases, the turning radius decreases. The rate of which the boats heading changes, also known as the angular velocity, $\omega$, can be represented as the differential of the angle and time: $\frac{dA}{dt}$ or as the velocity over the radius: $\frac{v}{R}$.

Therefore $\omega = \frac{v}{(K|\theta|)^-1}$, and so: $$\omega = vK|\theta|$$

We can simply intergrate this to find the change in angle over time to give : $$ \Delta\omega = vK|\theta|t $$

However we understand that the boat does not turn to the target radius immediately, there is always some lag between turning the rudder and the target radius being reached.


The turn rate follows an exponential decay until it reaches the target, the standard decay equation is $ N_{t+1} = N_{t} - e^{-\lambda t} $

Where lambda is our decay constant and t is our discrete time value.


This can be implemented with our boat applicable values: $$ \omega(t) = \omega_{\infty} \left( 1 - e^{-t/\tau} \right) $$

This algorithm is built into the code as descibed in the technical solution (boatSim.py)

#### Psuedo Code

The following psuedo code discrete time approximation of the boats response to the tiller:

[PSUEDO CODE GOES HERE]

#### Disturbances

The disturbance code randomly picks a start time based on the current time by adding a random value. Its duration is determined by repeating this process on the start time. When a disturbance is inflicted on the simulator its rate of turn is increased or decreased by a random magnitude. All random values are determined within a realistic range to mimic the affect of sea conditions on a boat.

#### Code


In [ ]:
TIMECONSTANT = 0.333333
MAX_DISTURBANCE_DURATION = 5000 # ms
MIN_DISTURBANCE_DURATION = 2000 # ms
MAX_DISTURBANCE_MAGNITUDE = 20 # degrees per secon


# Create a disturbance based on a random start time, duration and magnitude
def createDisturbance(self):
        currentTimeMilli = int(round(time.time() * 1000))
        
        disturbance =  random.randint(-MAX_DISTURBANCE_MAGNITUDE, MAX_DISTURBANCE_MAGNITUDE)
        
        startTime = currentTimeMilli + random.randint( MAX_DISTURBANCE_DURATION, 2 * MAX_DISTURBANCE_DURATION)
        
        endTime = startTime + random.randint(MIN_DISTURBANCE_DURATION,MAX_DISTURBANCE_DURATION)
        
        output = {'endTime': endTime,
                'disturbance': disturbance,
                'startTime': startTime}
        
        return output
    
# Return the disturbance if it is currently active, otherwise create a new one
def disturbance(self):
        currentTimeMilli = int(round(time.time() * 1000))
        if (currentTimeMilli < self.nextDisturbance['startTime']):
            return 0
        elif currentTimeMilli > self.nextDisturbance['startTime'] and currentTimeMilli < self.nextDisturbance['endTime']:
            return self.nextDisturbance['disturbance']
        else:
            self.nextDisturbance = self.createDisturbance()
            return 0



### PID Control Algorithm

The PID control algorithm allows SLAP to adjust the tiller angle to correct its course usiong the difference between its actual heading and the desired heading, this is how the auto pilot functions.

#### Mathmatics

The PID (Proportional, Integral, Derivative) control algorithm is a feedback control system that calculates an error value as the difference between a desired setpoint and a measured process variable. The algorithm applies three types of control:

1. Proportional (P): Responds proportionally to the current error
2. Integral (I): Accumulates past errors to eliminate steady-state error
3. Derivative (D): Predicts future error based on its rate of change

The output is calculated as:
$ output = K_p e_t + K_i \int e_t dt + K_d \frac{de}{dt} $

Where:
- $ (K_p), (K_i), (K_d) $ are tuning parameters
- $ (e_t) $ is the error at time t
- The integral term sums past errors
- The derivative term predicts future errors

This algorithm is implemented in SLAP to maintain the boat's heading by continuously adjusting the rudder angle based on the difference between the desired and actual heading.

#### Code

A simple PID algorithm is shown below and was used in the design stage to test the theory. The full PID algorithm used in the SLAP is detailed in the Technical Solution section.

In [4]:
# %load slap/src/pid/experimentForReport/pidModule.py

class PidController:

    def __init__(self,KP,KI,KD):
        self.kp = KP
        self.ki = KI
        self.kd = KD
        self.accumlatedError = 0
        self.lastPos = 0
        self.elapsed = 0

    def pid(self, pos, target, dt):

        # PROPORTIONAL-------
        error = target - pos
        proportional = error

        # Intergral----------
        intergal = ((error * dt) + self.accumlatedError)
        self.accumlatedError = intergal


        # Differential-------
        dpos = self.lastPos - pos
        differential = (dpos / dt)

        self.lastPos = pos

        return self.kp * proportional + self.ki * intergal + self.kd * differential

### Working with compass headings

There are complications in the arithmatic relating to compass headings. This is due to the fact a compass functions on a 360 degree basis. For example the difference between a heading of 5 degrees and 355 degrees should be -10 degrees and the 355 degrees + 10 degrees must equal 5 degrees not 365 degrees.

When calculating which way to adjust the tiller as a part of the control algorithm, this arthmetic complication is important to note otherwise the boat will attempt can attempt to correct its course via an longer arc than necessary.

The basic approach to dealing with compass headings is to use modulo 360 arithmetic. Secondly when calculating differences for control we must find the smallest change in heading necessary.

The following code provides each step to find the shortest path in modulo 360 form.

In [ ]:

# Get the error between the target and heading in its shortest path
def getHeadingError(self, target, heading):
    if target - heading <= 180:
        error = target - heading
    else:
        error = (target - heading) - 360
    return error

# Convert a heading to a value between 0 and 359
def compassify(input):
    print("Compassing: ", input)
    
    return input% 360



## Interface Design




## Security

The system will not be able to be accessed by other devices as it is local to the boat, this means that a security system will not be necessary


## Testing Strategy

